images - line by line, cmaera - col by col
    Q: is this hardware specific?
screen waits 1 frame cycle, camera waits 1 frame cycle 
    Q: r they the same?


two kinds of challenges:
    background challenge: one color
    lighting challenge: belt of different colr from background color
        belt is "lighting area"

ROI: region that camera is scanning when the screen is displaying the lighting area

In [2]:
import numpy as np
import cv2 
from sklearn.linear_model import LinearRegression as lr 
import pandas as pd
from deepface import DeepFace
import os
import shutil

In [4]:
input_video = "blake_video.mp4" # Replace with path of input video
output_directory1 = "output_1"
frames = "videoframes_raw"

# Video processing

In [ ]:
# Split video into frames in the videoframes directory
# Runtime: ~60 seconds

# Delete directory if it already exists and make a new one
if os.path.exists(output_directory1):
   shutil.rmtree(output_directory1)
os.makedirs(output_directory1)

vidcap = cv2.VideoCapture(input_video)
fps = vidcap.get(cv2.CAP_PROP_FPS) # get fps
frametime_list = []
print("FPS", fps)

if vidcap.isOpened():
  frame_count = 0

  while True:
      ret, frame = vidcap.read()
      
      if not ret:
          break
      
      # Get the timestamp of the current frame in milliseconds -> TODO: currently unused 
      timestamp = vidcap.get(cv2.CAP_PROP_POS_MSEC)
      frametime_list.append(timestamp)
      
      # Save each frame as an image in the output folder
      frame_name = f"frame_{frame_count:04d}.jpg"
      frame_path = os.path.join(output_directory1, frame_name)
      cv2.imwrite(frame_path, frame)

      frame_count += 1

  vidcap.release()

  frametime_list = np.array(frametime_list)

  print(f"Video frames saved in '{output_directory1}'")

In [ ]:
#CONVERT FRAMES INTO CROPPED FRAMES 
if os.path.exists(frames):
   shutil.rmtree(frames)
os.makedirs(frames)

for filename in sorted(os.listdir(output_directory1)):
    frame_path = os.path.join(output_directory1, filename)
    
    if os.path.isfile(frame_path):  # Check if it's a file (not a subdirectory)
        #face detection and alignment
        img = cv2.imread(frame_path)

        try: #skip frame if can't detect face
            face_objs = DeepFace.extract_faces(img_path = frame_path, 
                    target_size = (224, 224)
            )
        except:
            print("Couldn't detect face in frame " + frame_path)

        face = face_objs[0]['facial_area']
        x,y,w,h = face['x'],face['y'],face['w'],face['h']

        img = img[y:y+h, x:x+w]

    # Save each frame as an image in the output folder
        frame_name = f"frame_{frame_count:04d}.jpg"
        frame_path = os.path.join(frames, filename)
        cv2.imwrite(frame_path, img)

# Eqn 2

In [26]:
def eqn_2(pixel, color1, color2, E):
    '''
    #INPUTS:
        pixel: a single pixel with all 3 color channels
        color1: background color being shown on screen
        color2: primary color being shown on screen (band)
        E: illuminance for all 3 channels 

    Confirm that I{c1}/I{c2} = E{c1}/E{c2} (where c1 and c2 are the 2 colors being shown on the screen)  
    '''
    iFraction = pixel[color1]/(pixel[color2]+1)
    eFraction = E[color1]/E[color2]
    epsilon = 0.05
    return iFraction- eFraction <= epsilon 

In [27]:
def verifyEqn2(color1, color2, E, img):
    # Apply Eqn 2 on every pixel between response of lighting challenge and background challenge
    count = 0
    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            consistent = eqn_2(img[r][c][:], color1, color2, E)
            if not consistent:
                count+=1
                # print("Not consistent!")
                # return

    return (count/(img.shape[0]*img.shape[1]))

In [28]:
#EQN2 CHECK:
image = cv2.imread('videoframes_cropped/frame_0002.jpg')
color1 = 0
color2 = 1
E = [234,251,0]
count = 0
inconsistent = verifyEqn2(color1, color2, E, image)
print(inconsistent)

0.306488037109375


# ROI

In [12]:
# colorChanges = pd.read_csv('backend.csv')
frameTimes = []

for i in colorChanges: #for each color change
    color1, color2, u, startTime 
    offset = # randomly select an offset 
    t_u = startTime+offset
    imgIndex = np.sum(frameTimes<t_u)-1 #find all frames capturing time less than t_u, select last one
    imgPath = ' ' + frameTimes[imgIndex]
    img = cv2.imread(imgPath)
    exposureTime =  frameTimes[imgIndex+1]-frameTimes[imgIndex]

    #calculate eqn2 
    verifyEqn2(color1, color2, E, img)
    #STOP HERE IF DOESNT PASS

    a,b = roi(t_u, startTime, exposureTime, img)
    #find the corresponding cv2 image that covers that time frame


fps = 30 #(camera)
# 1/120 ms to draw one frame on the screen
""" 
starting w/ frontend's csv, select a random time t_u, 
input t_u, ct_k which is the start time of that color, 
check what colors are appearing at that time, input that into ROI 
"""
def roi(t_u, ct_k, ct_frame, image):
    '''
    INPUTS:
        t_u = time that this color started
        u = top of band
        ct_k = start time to exposure the first column of k-th capture frame
            --> find w/ firstImg
        ct_frame = exposure time of one captured frame 
            --> average time of each frame ? maybe can calculate w/ dict. 
        image = first image whose recording period covers t_u
    '''
    cols = image.shape[1]
    a = cols * (t_u - ct_k)/ct_frame
    b = a+0.2*image.shape[1]
    return [a,b]

In [15]:
#TESTING ROI
image = cv2.imread('videoframes_cropped/frame_0002.jpg')
t_u = 3
ct_k = 1
ct_frame = 30
roi(t_u, ct_k, ct_frame, image)

[17.066666666666666, 68.26666666666667]

In [16]:
image.shape

(256, 256, 3)

In [19]:
def models():
    # regression model
    a,b=10,50
    ROI = (a,b)
    
    imgWidth = image.shape[1]

    y_hat = 0
    if a < 0.25*imgWidth:
        avgCols = np.mean(image[:, a:25, :]) # take average value of columns [a,25] 
                ## ACROSS RGB ??
        output = lr1.predict(avgCols) #run regression on model 1
        y_hat += (0.25*imgWidth-a)*output
    if b < 0.5*imgWidth:
        #model2: avg. value of columns [25, b]
        # run regression on model 2 
        avgCols = None 
        output = lr2.predict(avgCols)
        y_hat += (b-0.25*imgWidth)*output

    if b < 0.75*imgWidth and a > 0.25*imgWidth:
        #model 3: avg value of columns [50,b]
        avgCols = None
        output = lr3.predict(avgCols)
        y_hat += (b-0.5*imgWidth)*output

    if b >=0.75*imgWidth:
        #model 4 : avg value of columns [75,b]
        avgCols = None
        output = lr4.predict(avgCols)
        y_hat += (b-0.75*imgWidth)*output
    
    y_hat = y_hat /imgWidth

    return y_hat


In [ ]:
# STEP 1: Calculate ROI
# calculate start time to show the lighting area, t_u
t_u = 0 #(u / rows) * t_frame
# find image - frame at t_u
img = frames[0] #[fps * t_u]

# calculate shift, l
l = 0 #cols * (t_u - ct_k) / (ct_frame)

u = 0.75 * img.shape[0]
rows = 0
t_frame = 0
cols = 0
ct_k = 0
ct_frame = 0
fps = 0
color1 = 0
color2 = 1
E = [234,251,0]

y_hat = roi(u, rows, t_frame, cols, ct_k, ct_frame, fps, color1, color2, E)
y_hat

In [ ]:
def criteria(frames):
    t_begin, u, rows, t_frame, cols, ct_k, ct_frame, fps, color1, color2, E = 1,2,3,4,5,6,7,8,9,10,11
    
    dVals = []
    for frame in frames:
        y_hat_i = roi(frames, t_begin, u, rows, t_frame, cols, ct_k, ct_frame, fps, color1, color2, E)

        # calc criteria
        d = 0.25 #% of screen the band is shown on 
        imgRows = frame.shape[0] 
        
        d_i = y_hat_i - (u + u+imgRows*0.25)/2
        dVals.append(d_i)

    mean = np.mean(dVals)
    var = np.var(dVals)

    threshold = -5
    return mean * np.sqrt(var) < np.exp(threshold)

IMPLEMENTATION

In [ ]:
#training linear regression models to be used to identify location of band of color
#TO DO: need data to train on !! 
lr1 = lr().fit(x1_train, y1_train)
lr2 = lr().fit(x2_train, y2_train)
lr3 = lr().fit(x3_train, y3_train)
lr4 = lr().fit(x4_train, y4_train)
